In [2]:
# Imports
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

from collections import Counter

import os, sys
from copy import deepcopy

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from IPython.display import clear_output 


import matplotlib.pyplot as plt


import torch
import networkx as nx
from networkx import pagerank
from networkx import set_node_attributes

from torch_geometric.data import Data 
from torch_geometric.data import Dataset
from torch_geometric import utils

In [3]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

from collections import Counter

In [ ]:
 !nvidia-smi -L

In [4]:
df = pd.read_csv("ccf.csv",sep=';')

# df = df.drop(['TX_DATETIME'], axis=1)
df = df.drop(['TERM_COUNTRY'], axis=1)
df.head(5)

df['TX_FRAUD'] = df['TX_FRAUD'].astype(int)
df['TX_ACCEPTED'] = df['TX_ACCEPTED'].astype(int)

In [ ]:
df.head()

In [5]:
df = df.copy()

In [ ]:
df.head()

In [9]:
df = df.copy()

In [6]:
df = df.drop(['Unnamed: 0'], axis=1)
df = df.drop(['TX_DATETIME'], axis=1)

In [ ]:
df.head()

In [8]:
df.shape

(3724348, 6)

In [9]:
df['TX_FRAUD'].value_counts()

TX_FRAUD
0    3700000
1      24348
Name: count, dtype: int64

In [10]:
df.isna().sum()

CARD_PAN_ID    0
TERM_MIDUID    0
TERM_MCC       0
TX_AMOUNT      0
TX_ACCEPTED    0
TX_FRAUD       0
dtype: int64

In [12]:
df.columns

Index(['CARD_PAN_ID', 'TERM_MIDUID', 'TERM_MCC', 'TX_AMOUNT', 'TX_ACCEPTED',
       'TX_FRAUD'],
      dtype='object')

In [ ]:
# printing info on the columns
df.info()

In [ ]:
df.describe()

In [ ]:
df['TX_AMOUNT'].head(5)


In [8]:

# Assuming df is your DataFrame and 'TX_AMOUNT' is the column you want to convert to float
# Remove commas from 'TX_AMOUNT' column and convert to float
df['TX_AMOUNT'] = df['TX_AMOUNT'].str.replace(',', '.').astype(float)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Initialize label encoders
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Encode the columns
df['CARD_PAN_ID'] = label_encoder1.fit_transform(df['CARD_PAN_ID'])
df['TERM_MIDUID'] = label_encoder2.fit_transform(df['TERM_MIDUID'])

# Ensure uniqueness
df['CARD_PAN_ID'] = df['CARD_PAN_ID'] + max(df['TERM_MIDUID']) + 1

# Display the DataFrame with encoded columns
print(df.head)


In [16]:
# Save the DataFrame to a CSV file
df.to_csv('new_IDs_ccf.csv', index=False)

In [ ]:
# printing info on the columns
df.info()

In [24]:
#type(df['CARD_PAN_ID'])

pandas.core.series.Series

In [ ]:
df['CARD_PAN_ID'].value_counts()

In [ ]:
df['TERM_MIDUID'].value_counts()

In [ ]:
df.nunique()

In [ ]:
df.info()

In [10]:
from sklearn.preprocessing import StandardScaler

# Create an instance of StandardScaler
scaler = StandardScaler()

# Specify the columns you want to scale
columns_to_scale = ['TERM_MCC']  # Replace with the actual column names

# Fit the scaler to your data
scaler.fit(df[columns_to_scale])

# Transform the data using the fitted scaler
scaled_data = scaler.transform(df[columns_to_scale])

# Replace the original columns with the scaled data
df[columns_to_scale] = scaled_data


In [ ]:
correlation_matrix = df.corr()
correlation_matrix

In [ ]:
plt.figure(figsize=(10, 10))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(correlation_matrix, vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12})

In [ ]:
pd.plotting.scatter_matrix(df[df.columns], figsize=(30, 22))

In [19]:
df.columns

Index(['CARD_PAN_ID', 'TERM_MIDUID', 'TERM_MCC', 'TX_AMOUNT', 'TX_ACCEPTED',
       'TX_FRAUD'],
      dtype='object')

In [21]:
attribute_cols = list(df.columns)
print(attribute_cols)
attribute_cols.remove('CARD_PAN_ID')
# attribute_cols.remove('AGENT_NAME')
attribute_cols.remove('TERM_MIDUID')
attribute_cols.remove('TX_FRAUD')

['CARD_PAN_ID', 'TERM_MIDUID', 'TERM_MCC', 'TX_AMOUNT', 'TX_ACCEPTED', 'TX_FRAUD']


In [22]:
attribute_cols

['TERM_MCC', 'TX_AMOUNT', 'TX_ACCEPTED']

In [23]:
import networkx as nx
from networkx import circular_layout, spectral_layout

def make_graph(df):
    G = nx.Graph()
    # nx.set_edge_attributes(G_to_draw, labels, "labels")
    for index, row in df.iterrows():
        set_node_attributes(G, row[attribute_cols].values, name=row['TERM_MIDUID'])
        G.add_edge(row['CARD_PAN_ID'], row['TERM_MIDUID'])
    return G

In [24]:
# Firstly, we are creating a small network with 20 rows so that we can draw it
G_to_draw = nx.Graph()
for index, row in df.head(90).iterrows():
    # G_to_draw.add_edge(row['ID'], row['AGENT_NAME'])
    G_to_draw.add_edge(row['CARD_PAN_ID'], row['TERM_MIDUID'])


In [ ]:
plt.figure(figsize=(10, 10))
nx.draw(G_to_draw, 
        node_size=1000, 
        with_labels=True)

In [ ]:
df.info()

In [ ]:
graph_df =  make_graph(df)

In [ ]:
pr = pagerank(graph_df, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, dangling=None)


In [ ]:
#df['pagerank_TERM_MCC'] = df['TERM_MCC'].apply(lambda x: pr[x])
#df['pagerank_TX_AMOUNT'] = df['TX_AMOUNT'].apply(lambda x: pr[x])
#df['pagerank_TX_ACCEPTED'] = df['TX_ACCEPTED'].apply(lambda x: pr[x])
df['pagerank_CARD_PAN_ID'] = df['CARD_PAN_ID'].apply(lambda x: pr[x])
df['pagerank_TERM_MIDUID'] = df['TERM_MIDUID'].apply(lambda x: pr[x])


In [ ]:
df.head()

other network features to use:
1- bayan (best community detection out there)
https://colab.research.google.com/drive/1_Clxp5FcEYJVn_w7Q6zm8bX0t1TEg7m_?usp=sharing
2- reconstruction of a network: (work of tiago in NetSci)
https://graph-tool.skewed.de/

In [ ]:
# def get_embeddings(graph):
    # extracting network measures to use as node attributes
    #degree=np.array(list(dict(graph.degree()).values())).reshape(-1,1) 
    #degree_centrality=np.array(list(dict(nx.degree_centrality(graph)).values())).reshape(-1,1)
    #closeness=np.array(list(dict(nx.closeness_centrality(graph)).values())).reshape(-1,1)
    #betweenness=np.array(list(dict(nx.betweenness_centrality(graph)).values())).reshape(-1,1)
    #clustering_coefficient=np.array(list(dict(nx.clustering(graph)).values())).reshape(-1,1)
    #pagerank = ncp.array(list(dict(nx.pagerank(graph)).values())).reshape(-1,1)
    #personalised_pagerank=np.array(list(dict(nx.pagerank(graph, personalization=nx.get_node_attributes(graph, 'fraud'))).values())).reshape(-1,1)
    #embeddings = np.concatenate((degree, closeness, betweenness,distance_instructor, distance_admin),axis=1)
   
    #embeddings = np.concatenate((degree, closeness, betweenness),axis=1)
    #embeddings = np.concatenate((degree_centrality, clustering_coefficient, pagerank),axis=1)
    # normalizing the attributes
    #scale = StandardScaler()
    #embeddings = scale.fit_transform(embeddings)#.reshape(-1,1))
    #return embeddings

In [ ]:
dc = nx.degree_centrality(G_full)

df['degree_centrality_id'] = df['ID'].apply(lambda x: dc[x])
df['degree_centrality_agent_name'] = df['AGENT_NAME'].apply(lambda x: dc[x])
df['degree_centrality_supervisor_issue'] = df['SUPERVISOR_ISSUE'].apply(lambda x: dc[x])
df['degree_centrality_place_issue'] = df['PLACE_ISSUED'].apply(lambda x: dc[x])

In [ ]:
y = df['TX_FRAUD']

In [103]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['CARD_PAN_ID'] = label_encoder.fit_transform(df['CARD_PAN_ID'])
df['TERM_MIDUID'] = label_encoder.fit_transform(df['TERM_MIDUID'])

Splitting now the df into train and val

In [ ]:
def split_train_val_test(df):
    # Assuming you have a DataFrame named df with a 'date' column

    # Convert the 'date' column to a datetime data type
    df['date'] = pd.to_datetime(df['TX_DATETIME'])

    # Define the date ranges for train, validation, and test
    # train_start = pd.to_datetime('2013-10-01')
    # train_end = pd.to_datetime('2013-10-25')
    # test_end = pd.to_datetime('2013-11-06')

    train_start = pd.to_datetime('2013-10-01')
    train_end = pd.to_datetime('2013-10-03')
    test_end = pd.to_datetime('2013-10-04')

    # Calculate the size of the validation set as 20% of the train set
    # train_size = int(len(df) * 0.8)
    # val_size = len(df) - train_size

    # Create boolean masks for train, validation, and test
    train_mask = (df['date'] >= train_start) & (df['date'] <= train_end)
    # val_mask = (df['date'] > train_end) & (df['date'] <= test_end)
    test_mask = (df['date'] > train_end) & (df['date'] <= test_end)

    # Apply the masks to select the rows
    train = df[train_mask]
    #val = df[val_mask]
    test = df[test_mask]

    # Print the resulting train, validation, and test DataFrames
    print("Train:")
    print(train)
    # print("\nValidation:")
    # print(val)
    print("\nTest:")
    print(test)

    return train, test

In [ ]:
train, test = split_train_val_test(df_w_node_feature)

In [ ]:
print("Number of nodes in the test graph: ", len(graph_test.nodes), " number of labels retrieved: ", len(test_labels))

In [24]:
train_labels = np.array([graph_train.nodes[i]['fraud']  for i in graph_train.nodes])
test_labels = np.array([graph_test.nodes[i]['fraud']  for i in graph_test.nodes])

In [ ]:
train_labels = np.array([graph_train.nodes[i]['fraud']  for i in graph_train.nodes])

In [ ]:
print("Number of nodes in the train graph: ", len(graph_train.nodes), " number of labels retrieved: ", len(train_labels))

In [ ]:
len(graph_train.edges)

In [ ]:
len(graph_test.nodes)

In [10]:
attribute_cols = list(df.columns)
print(attribute_cols)
attribute_cols.remove('CARD_PAN_ID')
# attribute_cols.remove('AGENT_NAME')
attribute_cols.remove('TERM_MIDUID')
attribute_cols.remove('TX_FRAUD')

['CARD_PAN_ID', 'TERM_MIDUID', 'TERM_MCC', 'TX_AMOUNT', 'TX_ACCEPTED', 'TX_FRAUD']


In [12]:
import networkx as nx
from networkx import circular_layout, spectral_layout

def make_graph(df):
    G = nx.Graph()
    # nx.set_edge_attributes(G_to_draw, labels, "labels")
    for index, row in df.iterrows():
        set_node_attributes(G, row[attribute_cols].values, name=row['TERM_MIDUID'])
        G.add_node(row['CARD_PAN_ID'],shape="square", fraud=row['TX_FRAUD'])
        G.add_node(row['TERM_MIDUID'], shape="circle", fraud=0)
        G.add_edge(row['CARD_PAN_ID'], row['TERM_MIDUID'])
    return G

In [13]:
graph_train =  make_graph(df.head(1000))

In [14]:
from sklearn.preprocessing import StandardScaler
def get_edge_index(graph):
    # create edge index
    adj = nx.to_scipy_sparse_array(graph).tocoo()
    row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
    col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
    edge_index = torch.stack([row, col], dim=0)
    return edge_index

def get_embeddings(graph):
    # extracting network measures to use as node attributes
    #degree=np.array(list(dict(graph.degree()).values())).reshape(-1,1) 
    degree_centrality=np.array(list(dict(nx.degree_centrality(graph)).values())).reshape(-1,1)
    #closeness=np.array(list(dict(nx.closeness_centrality(graph)).values())).reshape(-1,1)
    #betweenness=np.array(list(dict(nx.betweenness_centrality(graph)).values())).reshape(-1,1)
    clustering_coefficient=np.array(list(dict(nx.clustering(graph)).values())).reshape(-1,1)
    pagerank = np.array(list(dict(nx.pagerank(graph)).values())).reshape(-1,1)
    #personalised_pagerank=np.array(list(dict(nx.pagerank(graph, personalization=nx.get_node_attributes(graph, 'fraud'))).values())).reshape(-1,1)
    #embeddings = np.concatenate((degree, closeness, betweenness,distance_instructor, distance_admin),axis=1)
   
    #embeddings = np.concatenate((degree, closeness, betweenness),axis=1)
    embeddings = np.concatenate((degree_centrality, clustering_coefficient, pagerank),axis=1)
    # normalizing the attributes
    scale = StandardScaler()
    embeddings = scale.fit_transform(embeddings)#.reshape(-1,1))
    return embeddings

In [15]:
train_labels = np.array([graph_train.nodes[i]['fraud']  for i in graph_train.nodes])

In [16]:
import time

In [17]:
%%time
edge_index_train = get_edge_index(graph_train)
#edge_index_test = get_edge_index(graph_test)

CPU times: user 13.6 ms, sys: 3.17 ms, total: 16.7 ms
Wall time: 15.2 ms


In [ ]:
edge_index_train

In [18]:
%%time
embeddings_train = get_embeddings(graph_train)

CPU times: user 44.8 ms, sys: 4.08 ms, total: 48.8 ms
Wall time: 50 ms


In [ ]:
embeddings_train

In [29]:
%%time

#embeddings_test = get_embeddings(graph_test)



CPU times: user 7.82 s, sys: 19.4 ms, total: 7.84 s
Wall time: 7.86 s


In [ ]:
# Making the Data objects
## Train
data_train = Data(edge_index=edge_index_train)
data_train.num_nodes = graph_train.number_of_nodes()
# labels
y_train = torch.from_numpy(train_labels).type(torch.long)
data_train.y = y_train.clone().detach()
data_train.num_classes = 2
'''
## Test
data_test = Data(edge_index=edge_index_test)
data_test.num_nodes = graph_test.number_of_nodes()
# labels
y_test = torch.from_numpy(test_labels).type(torch.long)
data_test.y = y_test.clone().detach()
data_test.num_classes = 2


In [19]:
# Making the Data objects
## Train
data_train = Data(edge_index=edge_index_train)
data_train.num_nodes = graph_train.number_of_nodes()
# labels
y_train = torch.from_numpy(train_labels).type(torch.long)
data_train.y = y_train.clone().detach()
data_train.num_classes = 2

In [20]:
# embedding train
data_train.x = torch.from_numpy(embeddings_train).type(torch.float32)
# embedding test
#data_test.x = torch.from_numpy(embeddings_test).type(torch.float32)

In [ ]:
data_test.x

In [ ]:
data_train.x

RUN UNTIL HERE

In [21]:
import pygod

In [22]:
from pyod.models.lof import LOF
from torch_geometric.nn import MLP

In [31]:
from pygod.detector import SCAN
from pygod.metric import eval_roc_auc

model = SCAN(eps=0.5, mu=2, contamination=0.1, verbose=0)
model.fit(data_train)
# get outlier scores on the training data (transductive setting)
score = model.decision_score_

In [ ]:
pred, score = model.predict(data_train, return_score=True)

In [ ]:
auc_score = eval_roc_auc(data_train.y, score)

print(" * score ", score)
print(" * pred ", pred)
print('AUC Score:', auc_score)

from pygod.detector import DOMINANT
from pygod.metric import eval_roc_auc

model = DOMINANT(hid_dim=64, num_layers=4, epoch=30, gpu=0)
model.fit(data)
# get outlier scores on the training data (transductive setting)
score = model.decision_score_
# predict labels and scores on the testing data (inductive setting)
#pred, score = model.predict(data_test, return_score=True)
#auc_score = eval_roc_auc(data_test.y, score)

print(" * score ", score)
'''print(" * pred ", pred)
print('AUC Score:', auc_score)

In [ ]:
from pygod.detector import DOMINANT
from pygod.metric import eval_roc_auc

model = DOMINANT(hid_dim=64, num_layers=4, epoch=30, gpu=0)
model.fit(data_train)
# get outlier scores on the training data (transductive setting)
score = model.decision_score_
# predict labels and scores on the testing data (inductive setting)
pred, score = model.predict(data_test, return_score=True)
auc_score = eval_roc_auc(data_test.y, score)

print(" * score ", score)
print(" * pred ", pred)
print('AUC Score:', auc_score)

In [2]:
from pygod.detector import DOMINANT
from pygod.metric import eval_roc_auc

In [ ]:
data.x.shape

In [ ]:
from pygod.detector import AdONE
from pygod.metric import eval_roc_auc

model = AdONE(hid_dim=64, num_layers=4,epoch=100)
model.fit(data_train)
# get outlier scores on the training data (transductive setting)
score = model.decision_score_



In [ ]:
# TODO: ADD A SHALLOW MODEL SUCH AS SCAN
from pygod.detector import DOMINANT, ANOMALOUS, GAAN, AnomalyDAE, GAE, AdONE, SCAN
from pygod.metric import eval_roc_auc

cuda = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.device_count())


model_list = {}
dominant_model = DOMINANT(hid_dim=64, num_layers=4, epoch=100, gpu=0)
anomalous_model = ANOMALOUS(gamma=1.0, weight_decay=0.0, lr=0.004, epoch=100, gpu=0, contamination=0.1, verbose=0)
gaan_model = GAAN(noise_dim=16, hid_dim=64, num_layers=4, contamination=0.1, lr=0.004, epoch=100, gpu=0)
anomalydae_model = AnomalyDAE(emb_dim=64, hid_dim=64, num_layers=4,epoch=100, gpu=0)
gae_model = GAE(hid_dim=64, num_layers=4, epoch=100)
adone_model = AdONE(hid_dim=64, num_layers=4,epoch=100, gpu=0)
scan_model = SCAN(eps=0.5, mu=2, contamination=0.1, verbose=0)

model_list["DOMINANT"] = dominant_model
model_list["ANOMALOUS"] = anomalous_model
model_list["GAAN"] = gaan_model
model_list["AnomalyDAE"] = anomalydae_model
model_list["GAE"] = gae_model
model_list["AdONE"] = adone_model
model_list["SCAN"] = scan_model




# TODO: CHANGE THIS TO USE THE TEST SET
for model_name, model in model_list.items():
    print("Model: ", model_name)
    model.fit(data)
    # get outlier scores on the training data (transductive setting)
    score = model.decision_score_
    # predict labels and scores on the testing data (inductive setting)
    #pred, score = model.predict(data_test, return_score=True)
    # auc_score = eval_roc_auc(data_test.y, score)
   
    #print(" * score ", score)
    #print(" * pred ", pred)
    # print('AUC Score:', auc_score)

4
Model:  DOMINANT
Model:  ANOMALOUS


In [101]:
print(torch.__version__)

2.0.1


In [ ]:
python -c "import torch; print(torch.__version__)"

In [ ]:
print(cuda.__version__)

In [67]:
data.y = data.y.bool()


In [66]:
data

Data(edge_index=[2, 1774], num_nodes=1199, num_classes=2, x=[1199, 3])

In [68]:
# train a dominant detector
from pygod.detector import DOMINANT

model = DOMINANT(num_layers=4, epoch=20)  # hyperparameters can be set here
model.fit(data)  # input data is a PyG data object

# get outlier scores on the training data (transductive setting)
score = model.decision_score_

# predict labels and scores on the testing data (inductive setting)
#pred, score = model.predict(test_data, return_score=True)

In [ ]:
score

In [38]:
y = data_train.y.bool()

In [39]:
y

tensor([False, False, False,  ..., False, False, False])

In [41]:
from pygod.detector import GAE

detector = GAE(hid_dim=64, num_layers=4, epoch=100)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([1.4246e-05, 1.4246e-05, 1.4246e-05,  ..., 1.4246e-05, 1.4246e-05,
        8.8046e-03])
Probability:
tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0185])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.3981171548117155


In [42]:
from pygod.detector import DOMINANT

detector = DOMINANT(hid_dim=64, num_layers=4, epoch=100, gpu=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.5005, 0.5005, 0.5005,  ..., 0.5005, 0.5005, 0.5058])
Probability:
tensor([0.0027, 0.0027, 0.0027,  ..., 0.0027, 0.0027, 0.0031])
Confidence:
tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 0.9537])
AUC Score: 0.6502092050209205


In [35]:
from pygod.detector import DOMINANT

detector = DOMINANT(hid_dim=256, dropout=0.1, num_layers=4, epoch=100, gpu=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.5023, 0.5023, 0.5023,  ..., 0.5023, 0.5023, 0.4993])
Probability:
tensor([0.0031, 0.0031, 0.0031,  ..., 0.0031, 0.0031, 0.0029])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.6523012552301255


In [39]:
from pygod.detector import DOMINANT

detector = DOMINANT(hid_dim=46, dropout=0.3, num_layers=4, epoch=1000, gpu=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.5070, 0.5070, 0.5070,  ..., 0.5070, 0.5070, 0.4984])
Probability:
tensor([0.0105, 0.0105, 0.0105,  ..., 0.0105, 0.0105, 0.0100])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.735878661087866


In [32]:
from pygod.detector import SCAN

detector = SCAN(eps=0.5, mu=2, contamination=0.1, verbose=0)


#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/scan.py:162: UserWarning: This detector is transductive only. Training from scratch with the input data.
  warnings.warn("This detector is transductive only. "


Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0., 0., 0.,  ..., 0., 0., 0.])
Probability:
tensor([nan, nan, nan,  ..., nan, nan, nan])
Confidence:
tensor([0., 0., 0.,  ..., 0., 0., 0.])
AUC Score: 0.5


In [34]:
from pygod.detector import SCAN

detector = SCAN(eps=0.8, mu=10, contamination=0.1, verbose=0)


#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/scan.py:162: UserWarning: This detector is transductive only. Training from scratch with the input data.
  warnings.warn("This detector is transductive only. "


Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0., 0., 0.,  ..., 0., 0., 0.])
Probability:
tensor([nan, nan, nan,  ..., nan, nan, nan])
Confidence:
tensor([0., 0., 0.,  ..., 0., 0., 0.])
AUC Score: 0.5


In [ ]:
from pygod.detector import GUIDE

detector = GUIDE(hid_a=64, hid_s=4, num_layers=4, dropout=0.0, weight_decay=0.0, 
                    backbone=MLP, alpha=0.5, contamination=0.1, lr=0.004, epoch=epochs, gpu=0, 
                    batch_size=0, num_neigh=3, graphlet_size=4, selected_motif=True, cache_dir=None, 
                    verbose=0, save_emb=False, compile_model=False)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

In [34]:
from pygod.detector import GAAN

detector = GAAN(noise_dim=8,
                    hid_dim=32,
                    weight_decay=0.01,
                    dropout=0.1,
                    lr=0.1,
                    epoch=2,
                    gpu=0,
                    weight=0,
                    batch_size=32,
                    num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/gaan.py:124: UserWarning: MLP in GAAN does not use neighbor information.
  warnings.warn('MLP in GAAN does not use neighbor information.')


Labels:
tensor([1, 1, 1,  ..., 1, 1, 0])
Raw scores:
tensor([0.8325, 0.8325, 0.8325,  ..., 0.8325, 0.8325, 0.0000])
Probability:
tensor([0.4472, 0.4472, 0.4472,  ..., 0.4472, 0.4472, 0.0000])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.7370292887029288


In [ ]:
from pygod.detector import Radar

detector = radar_model = Radar(gamma=1.0, weight_decay=0.01, lr=0.1, epoch=2, gpu=0, contamination=0.1, verbose=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

In [36]:
from pygod.detector import DONE

detector =  DONE(hid_dim=64, num_layers=4, dropout=0.1, 
                  weight_decay=0.01, backbone=MLP, w1=0.2, w2=0.2, w3=0.2, w4=0.2, w5=0.2, 
                  contamination=0.1, lr=0.004, epoch=2, gpu=0, batch_size=64, num_neigh=3, verbose=0, save_emb=False, compile_model=False)



#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/done.py:137: UserWarning: Backbone is not used in AdONE.
  warnings.warn("Backbone is not used in AdONE.")
/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/done.py:217: UserWarning: This detector is transductive only. Training from scratch with the input data.
  warnings.warn("This detector is transductive only. "


Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.0007, 0.0006, 0.0006,  ..., 0.0078, 0.0081, 0.0137])
Probability:
tensor([0.0008, 0.0004, 0.0004,  ..., 0.0280, 0.0294, 0.0506])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.3943514644351464


In [ ]:
from pygod.detector import ANOMALOUS

detector = ANOMALOUS(gamma=1.0, weight_decay=0.0, lr=0.004, epoch=2, gpu=0, contamination=0.1, verbose=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

In [29]:
from pygod.detector import AnomalyDAE

detector = AnomalyDAE(emb_dim=64, hid_dim=64, num_layers=4,epoch=100, gpu=0)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 1])
Raw scores:
tensor([ 7.5695,  7.5695,  7.5695,  ...,  7.5695,  7.5695, 11.5881])
Probability:
tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2742])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.27918410041841


In [30]:
from pygod.detector import AnomalyDAE

detector = AnomalyDAE(emb_dim=64, hid_dim=64,batch_size = 64, lr=0.004, alpha=0.8, num_layers=4,epoch=5, gpu=0, num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([4.1430, 4.1430, 4.1430,  ..., 4.3615, 4.3615, 4.4246])
Probability:
tensor([0.0161, 0.0161, 0.0161,  ..., 0.0596, 0.0596, 0.0722])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.4044979079497908


In [23]:
from pygod.detector import CONAD

detector = CONAD(emb_dim=64, hid_dim=64,batch_size = 64, lr=0.004, alpha=0.8, num_layers=4,epoch=5, gpu=0, num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.4964, 0.4964, 0.4964,  ..., 0.4946, 0.4946, 0.5041])
Probability:
tensor([0.0017, 0.0017, 0.0017,  ..., 0.0015, 0.0015, 0.0022])
Confidence:
tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 0.8732])
AUC Score: 0.6771966527196652


In [27]:
from pygod.detector import CONAD

detector = CONAD(emb_dim=64, hid_dim=256,batch_size = 64, lr=0.004, alpha=0.8, num_layers=4,epoch=5, gpu=0, num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.5008, 0.5008, 0.5008,  ..., 0.4990, 0.4990, 0.4941])
Probability:
tensor([0.0026, 0.0026, 0.0026,  ..., 0.0025, 0.0025, 0.0022])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.8297071129707113


In [31]:
from pygod.detector import CONAD

detector = CONAD(emb_dim=64, hid_dim=256,batch_size = 64, lr=0.01, alpha=0.8, num_layers=4,epoch=5, gpu=0, num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.5001, 0.5001, 0.5001,  ..., 0.4994, 0.4994, 0.4983])
Probability:
tensor([0.0027, 0.0027, 0.0027,  ..., 0.0026, 0.0026, 0.0025])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.6688284518828452


In [29]:
from pygod.detector import GAAN

detector = GAAN(noise_dim=8,
                    hid_dim=256,
                    weight_decay=0.01,
                    dropout=0.1,
                    lr=0.1,
                    epoch=2,
                    gpu=0,
                    weight=0,
                    batch_size=64,
                    num_neigh=3)

#######################################################################
# Training
# --------
# To train the detector with the loaded data, simply feed the
# ``torch_geometric.data.Data`` object into the detector via ``fit``.


detector.fit(data_train)

#######################################################################
# Inference
# ---------
# After training, the detector is ready to use. You can use the detector
# to predict the labels, raw outlier scores, probability of the
# outlierness, and prediction confidence. Here, we use the loaded data
# as an example.


pred, score, prob, conf = detector.predict(data_train,
                                           return_pred=True,
                                           return_score=True,
                                           return_prob=True,
                                           return_conf=True)
print('Labels:')
print(pred)

print('Raw scores:')
print(score)

print('Probability:')
print(prob)

print('Confidence:')
print(conf)

#######################################################################
# Evaluation
# ----------
# To evaluate the performance outlier detector with AUC score, you can:


from pygod.metric import eval_roc_auc

auc_score = eval_roc_auc(data_train.y.bool(), score)
print('AUC Score:', auc_score)

/home/aidasa/bigdata/lib/python3.9/site-packages/pygod/detector/gaan.py:124: UserWarning: MLP in GAAN does not use neighbor information.
  warnings.warn('MLP in GAAN does not use neighbor information.')


Labels:
tensor([0, 0, 0,  ..., 0, 0, 0])
Raw scores:
tensor([0.8247, 0.8247, 0.8247,  ..., 0.8247, 0.8247, 0.0000])
Probability:
tensor([0.4454, 0.4454, 0.4454,  ..., 0.4454, 0.4454, 0.0000])
Confidence:
tensor([1., 1., 1.,  ..., 1., 1., 1.])
AUC Score: 0.8023012552301255
